# This code generate a Flask Web APP to predict and classify images, detecting if there's Violence or not

This codes includes function to create thumbnails of the videos in the dataset.

It uses the models generated on the Train_violence_model.ipynb file

In [32]:
# Install necesary libs
!pip install Flask
!pip install imgaug

   ---------------------------------------- 0.0/948.0 kB ? eta -:--:--
    -------------------------------------- 20.5/948.0 kB 320.0 kB/s eta 0:00:03
   - ------------------------------------- 41.0/948.0 kB 487.6 kB/s eta 0:00:02
   ---- --------------------------------- 112.6/948.0 kB 930.9 kB/s eta 0:00:01
   ----- -------------------------------- 143.4/948.0 kB 847.9 kB/s eta 0:00:01
   ----------- ---------------------------- 276.5/948.0 kB 1.2 MB/s eta 0:00:01
   ---------------------- ----------------- 532.5/948.0 kB 2.1 MB/s eta 0:00:01
   --------------------------- ------------ 645.1/948.0 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------  931.8/948.0 kB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 948.0/948.0 kB 2.6 MB/s eta 0:00:00


In [46]:
# Used once for creating thumbnails of videos
import os
import subprocess

# Specify the directory containing your videos
video_directory = './archive/Real Life Violence Dataset/NonViolence'

# Specify the directory where you want to save the thumbnails
thumbnail_directory = './archive/Real Life Violence Dataset/NonViolence/thumbnails'

# Ensure the thumbnail directory exists
if not os.path.exists(thumbnail_directory):
    os.makedirs(thumbnail_directory)

# Iterate over each file in the video directory
for filename in os.listdir(video_directory):
    # Check if the file is a video (you might need to adjust the condition based on your video formats)
    if filename.endswith(('.mp4', '.avi', '.mkv')):
        # Construct the full path to the video file
        video_path = os.path.join(video_directory, filename)
        
        # Construct the output path for the thumbnail
        thumbnail_path = os.path.join(thumbnail_directory, filename + '.jpg')
        
        # Use FFmpeg to generate a thumbnail from the video
        # This command extracts a frame at 10 seconds into the video and saves it as a JPEG
        subprocess.call(['ffmpeg', '-ss', '00:00:1.000', '-i', video_path, '-vframes', '1', thumbnail_path])

print("Thumbnails generated successfully.")

Thumbnails generated successfully.


In [33]:
# Function to Extract frames from videos
import cv2
import os
import imageio
import imgaug.augmenters as iaa
import imgaug as ia

IMG_SIZE = 128
ColorChannels = 3

def video_to_frames(video):
    vidcap = cv2.VideoCapture(video)

    import math
    rate = math.floor(vidcap.get(3))
    count = 0

    ImageFrames = []
    while vidcap.isOpened():
        ID = vidcap.get(1)
        success, image = vidcap.read()

        if success:
            # skipping frames to avoid duplications
            if (ID % 7 == 0):
                flip = iaa.Fliplr(1.0)
                zoom = iaa.Affine(scale=1.3)
                random_brightness = iaa.Multiply((1, 1.3))
                rotate = iaa.Affine(rotate=(-25, 25))

                image_aug = flip(image = image)
                image_aug = random_brightness(image = image_aug)
                image_aug = zoom(image = image_aug)
                image_aug = rotate(image = image_aug)

                rgb_img = cv2.cvtColor(image_aug, cv2.COLOR_BGR2RGB)
                resized = cv2.resize(rgb_img, (IMG_SIZE, IMG_SIZE))
                ImageFrames.append(resized)

            count += 1
        else:
            break

    vidcap.release()

    return ImageFrames

In [38]:
#Load trained model
from keras.models import load_model

model = load_model('model.h5')

In [142]:
from flask import Flask, render_template, request, Response
import os
import random

thumbnails = os.path.join('static', 'Violence','thumbnails')
videos = os.path.join('static', 'Violence',)
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = thumbnails

@app.route('/')
def list_videos():
    #List random file from directory
    video_directory = './static/Violence'
    all_files = os.listdir(video_directory)
    video_files = [f for f in all_files if f.endswith(('.mp4', '.avi', '.mkv'))]
    random_videos = random.sample(video_files, min(1, len(video_files)))
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], random_videos[0] + '.jpg')
    full_filename_video = os.path.join(videos, random_videos[0])

    image_file = video_to_frames(full_filename)[0]
    im = Image.fromarray(image_file)
    im.save("inference.jpeg")
    
    img_width, img_height = 128, 128
    img = image.load_img("inference.jpeg", target_size = (img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    return render_template('videos.html', videos=random_videos, images = full_filename, detection = model.predict(img))

@app.route('/select_video', methods=['POST'])
def select_video():
    #List random file from directory
    video_directory = './static/Violence'
    all_files = os.listdir(video_directory)
    video_files = [f for f in all_files if f.endswith(('.mp4', '.avi', '.mkv'))]
    random_videos = random.sample(video_files, min(1, len(video_files)))
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], random_videos[0] + '.jpg')
    full_filename_video = os.path.join(videos, random_videos[0])

    image_file = video_to_frames(full_filename)[0]
    im = Image.fromarray(image_file)
    im.save("inference.jpeg")
    
    img_width, img_height = 128, 128
    img = image.load_img("inference.jpeg", target_size = (img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    return render_template('videos.html', videos=random_videos, images = full_filename, detection = model.predict(img))

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2024 22:52:50] "GET /select_video HTTP/1.1" 405 -


1/1 [==============================] - 0s 25ms/step


127.0.0.1 - - [20/Apr/2024 22:52:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 22:52:55] "GET /static/Violence/thumbnails/V_42.mp4.jpg HTTP/1.1" 200 -


1/1 [==============================] - 0s 24ms/step


127.0.0.1 - - [20/Apr/2024 22:52:57] "POST /select_video HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 22:52:57] "GET /static/Violence/thumbnails/V_513.mp4.jpg HTTP/1.1" 200 -


...

127.0.0.1 - - [20/Apr/2024 22:52:59] "POST /select_video HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 22:52:59] "GET /static/Violence/thumbnails/V_879.mp4.jpg HTTP/1.1" 200 -


1/1 [==============================] - 0s 24ms/step


127.0.0.1 - - [20/Apr/2024 22:53:01] "POST /select_video HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 22:53:01] "GET /static/Violence/thumbnails/V_6.mp4.jpg HTTP/1.1" 200 -


1/1 [==============================] - 0s 24ms/step


127.0.0.1 - - [20/Apr/2024 22:53:05] "POST /select_video HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 22:53:05] "GET /static/Violence/thumbnails/V_155.mp4.jpg HTTP/1.1" 304 -
